In [1]:
import sqlparse
import csv
import re
from collections import defaultdict
from sql_metadata import Parser
import pprint

from column_usage import get_column_usage_from_logs, parse_simple_logs, get_where_usage
from index_generation import generate_all_indexes, prune_indexes, generate_create_index_commands

# (i) Epinions

### Scale factor = 1000
```
 Schema |     Name     | Type  |    Owner     | Persistence | Access method |  Size   | Description 
--------+--------------+-------+--------------+-------------+---------------+---------+-------------
 public | observations | table | project1user | permanent   | heap          | 5746 MB | 
 public | sessions     | table | project1user | permanent   | heap          | 784 kB  | 
 public | sources      | table | project1user | permanent   | heap          | 208 kB  | 
 public | types        | table | project1user | permanent   | heap          | 1464 kB | 
```

### Without index
```
{
 "scalefactor": "100",
 "Current Timestamp (milliseconds)": 1645652423106,
 "Benchmark Type": "epinions",
 "isolation": "TRANSACTION_READ_COMMITTED",
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 19.59045481317814,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 161233,
  "Maximum Latency (microseconds)": 165597,
  "Median Latency (microseconds)": 47476,
  "Minimum Latency (microseconds)": 10717,
  "25th Percentile Latency (microseconds)": 19026,
  "90th Percentile Latency (microseconds)": 77681,
  "99th Percentile Latency (microseconds)": 162937,
  "75th Percentile Latency (microseconds)": 57150,
  "Average Latency (microseconds)": 51774
 },
 "Throughput (requests/second)": 19.262581092455495
}
```

In [ ]:
log_file_path = "data/epinions_scale_100_rate_unlimited_time_60_simple.csv"
with open(log_file_path) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    log_file_dataframe = list(map(lambda x : x[13], reader))

In [ ]:
epinions_parsing_success, epinions_column_usage, epinions_where_predicates = parse_simple_logs(log_file_dataframe)

In [ ]:
pprint.pprint(epinions_parsing_success)

In [ ]:
pprint.pprint(epinions_column_usage)

In [ ]:
pprint.pprint(epinions_where_predicates)

### Adding single indices:

`CREATE INDEX idx_rewview_iid ON review (i_id);`

`CREATE INDEX idx_trust_sid ON trust (source_u_id);`

`CREATE INDEX idx_trust_tid ON trust (target_u_id);`

`CREATE INDEX idx_review_uid ON review (u_id);`

`CREATE INDEX idx_item_iid ON item (i_id);`

`CREATE INDEX idx_useracct_uid ON useracct (u_id);`


```
{
 "scalefactor": "100",
 "Current Timestamp (milliseconds)": 1645655337931,
 "Benchmark Type": "epinions",
 "isolation": "TRANSACTION_READ_COMMITTED",
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1997.9683018947728,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1211,
  "Maximum Latency (microseconds)": 34752,
  "Median Latency (microseconds)": 322,
  "Minimum Latency (microseconds)": 109,
  "25th Percentile Latency (microseconds)": 222,
  "90th Percentile Latency (microseconds)": 1069,
  "99th Percentile Latency (microseconds)": 1859,
  "75th Percentile Latency (microseconds)": 739,
  "Average Latency (microseconds)": 505
 },
 "Throughput (requests/second)": 1969.9516633065623
}
```

In [ ]:
generate_create_index_commands(prune_indexes(generate_all_indexes(epinions_column_usage, epinions_where_predicates)))

### Adding multi indices:

`CREATE INDEX idx_trust_comb ON trust (source_u_id, target_u_id);`

`CREATE INDEX idx_review_comb ON review (i_id, u_id);`

`CREATE INDEX idx_item_iid ON item (i_id);`

`CREATE INDEX idx_useracct_uid ON useracct (u_id);`

`CREATE INDEX idx_review_uid ON review (u_id);`

`CREATE INDEX idx_trust_tid ON trust (target_u_id);`

```
{
 "scalefactor": "100",
 "Current Timestamp (milliseconds)": 1645657598501,
 "Benchmark Type": "epinions",
 "isolation": "TRANSACTION_READ_COMMITTED",
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 2024.5618091032316,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1226,
  "Maximum Latency (microseconds)": 35793,
  "Median Latency (microseconds)": 312,
  "Minimum Latency (microseconds)": 105,
  "25th Percentile Latency (microseconds)": 211,
  "90th Percentile Latency (microseconds)": 1070,
  "99th Percentile Latency (microseconds)": 1902,
  "75th Percentile Latency (microseconds)": 733,
  "Average Latency (microseconds)": 499
 },
 "Throughput (requests/second)": 1991.4621610037952
}
```

# (ii) Jungle

### Scale factor = 1000
```
 Schema |     Name      | Type  |    Owner     | Persistence | Access method |  Size   | Description 
--------+---------------+-------+--------------+-------------+---------------+---------+-------------
 public | jungle        | table | project1user | permanent   | heap          | 6011 MB | 
```

### Without index

```
{
    "scalefactor": "1000",
    "Current Timestamp (milliseconds)": 1645468671728,
    "Benchmark Type": "indexjungle",
    "isolation": null,
    "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
    "Goodput (requests/second)": 1.6112963871000103,
    "terminals": "1",
    "DBMS Type": "POSTGRES",
    "Latency Distribution": {
        "95th Percentile Latency (microseconds)": 632282,
        "Maximum Latency (microseconds)": 646268,
        "Median Latency (microseconds)": 623239,
        "Minimum Latency (microseconds)": 607371,
        "25th Percentile Latency (microseconds)": 619141,
        "90th Percentile Latency (microseconds)": 630341,
        "99th Percentile Latency (microseconds)": 644008,
        "75th Percentile Latency (microseconds)": 627854,
        "Average Latency (microseconds)": 623411
    },
    "Throughput (requests/second)": 1.601329605324134
}
```

In [ ]:
log_file_path = "./data/jungle_scale_1000_rate_unlimited_time_600_simple.csv"
with open(log_file_path) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    log_file_data = list(map(lambda x : x[13], reader))
jungle_parsing_success, jungle_column_usage, jungle_where_predicates = parse_simple_logs(log_file_data)

In [ ]:
pprint.pprint(jungle_parsing_success)

In [ ]:
pprint.pprint(jungle_column_usage)

In [ ]:
pprint.pprint(jungle_where_predicates)

In [ ]:
generate_create_index_commands(prune_indexes(generate_all_indexes(jungle_column_usage, jungle_where_predicates)))

### Added index: 

`CREATE INDEX idx_jungle_int_field3 ON jungle (int_field3);`
```
{
 "scalefactor": "1000",
 "Current Timestamp (milliseconds)": 1645469508485,
 "Benchmark Type": "indexjungle",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1383.0397270709402,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1251,
  "Maximum Latency (microseconds)": 7005,
  "Median Latency (microseconds)": 711,
  "Minimum Latency (microseconds)": 150,
  "25th Percentile Latency (microseconds)": 435,
  "90th Percentile Latency (microseconds)": 1173,
  "99th Percentile Latency (microseconds)": 1408,
  "75th Percentile Latency (microseconds)": 997,
  "Average Latency (microseconds)": 722
 },
 "Throughput (requests/second)": 1382.1797272406525
}
```

# (iii) Timeseries

### Scale factor = 10
```
 Schema |     Name     | Type  |    Owner     | Persistence | Access method |  Size   | Description 
--------+--------------+-------+--------------+-------------+---------------+---------+-------------
 public | observations | table | project1user | permanent   | heap          | 5746 MB | 
 public | sessions     | table | project1user | permanent   | heap          | 784 kB  | 
 public | sources      | table | project1user | permanent   | heap          | 208 kB  | 
 public | types        | table | project1user | permanent   | heap          | 1464 kB | 
```

### Without index
```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645546729938,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 0.591362724923002,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1884720,
  "Maximum Latency (microseconds)": 1931072,
  "Median Latency (microseconds)": 1687350,
  "Minimum Latency (microseconds)": 1553243,
  "25th Percentile Latency (microseconds)": 1659923,
  "90th Percentile Latency (microseconds)": 1845196,
  "99th Percentile Latency (microseconds)": 1921746,
  "75th Percentile Latency (microseconds)": 1726973,
  "Average Latency (microseconds)": 1705219
 },
 "Throughput (requests/second)": 0.5847181999238672
}
```

In [ ]:
log_file_path = "./data/timeseries_scale_10_rate_unlimited_time_300_simple.csv"
with open(log_file_path) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    log_file_data = list(map(lambda x : x[13], reader))
timeseries_parsing_success, timeseries_column_usage, timeseries_where_predicates = parse_simple_logs(log_file_data)

In [ ]:
pprint.pprint(timeseries_parsing_success)

In [ ]:
pprint.pprint(timeseries_column_usage)

In [ ]:
pprint.pprint(timeseries_where_predicates)

### Adding single indices:

`CREATE INDEX idx_observations_created_time ON observations (created_time);`

`CREATE INDEX idx_observations_session_id ON observations (session_id);`

`CREATE INDEX idx_observations_source_id ON observations (source_id);`

`CREATE INDEX idx_observations_type_id ON observations (type_id);`

```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645555654179,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 25.843852375660493,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 66818,
  "Maximum Latency (microseconds)": 86139,
  "Median Latency (microseconds)": 41596,
  "Minimum Latency (microseconds)": 4061,
  "25th Percentile Latency (microseconds)": 29577,
  "90th Percentile Latency (microseconds)": 63509,
  "99th Percentile Latency (microseconds)": 74320,
  "75th Percentile Latency (microseconds)": 53250,
  "Average Latency (microseconds)": 38786
 },
 "Throughput (requests/second)": 25.764118160849353
}
```

In [ ]:
generate_create_index_commands(prune_indexes(generate_all_indexes(timeseries_column_usage, timeseries_where_predicates)))


### Adding multiindex:

`CREATE INDEX idx_observations_comb ON observations (source_id, session_id, type_id);`

```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645556187951,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1514.628448292875,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1157,
  "Maximum Latency (microseconds)": 5221,
  "Median Latency (microseconds)": 607,
  "Minimum Latency (microseconds)": 230,
  "25th Percentile Latency (microseconds)": 459,
  "90th Percentile Latency (microseconds)": 1038,
  "99th Percentile Latency (microseconds)": 1346,
  "75th Percentile Latency (microseconds)": 812,
  "Average Latency (microseconds)": 660
 },
 "Throughput (requests/second)": 1512.1035304469679
}
```

### Adding index on sorted field:

\+ `CREATE INDEX idx_observations_created_time ON observations (created_time);`

```
{
 "scalefactor": "10",
 "Current Timestamp (milliseconds)": 1645556588675,
 "Benchmark Type": "timeseries",
 "isolation": null,
 "DBMS Version": "PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit",
 "Goodput (requests/second)": 1482.5532255022954,
 "terminals": "1",
 "DBMS Type": "POSTGRES",
 "Latency Distribution": {
  "95th Percentile Latency (microseconds)": 1176,
  "Maximum Latency (microseconds)": 4890,
  "Median Latency (microseconds)": 620,
  "Minimum Latency (microseconds)": 236,
  "25th Percentile Latency (microseconds)": 473,
  "90th Percentile Latency (microseconds)": 1056,
  "99th Percentile Latency (microseconds)": 1371,
  "75th Percentile Latency (microseconds)": 826,
  "Average Latency (microseconds)": 674
 },
 "Throughput (requests/second)": 1480.0398923517653
}
```

In [2]:
log_file_path = "./data/jungle_live.csv"
with open(log_file_path) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    log_file_data = list(map(lambda x : x[13], reader))
jungle_parsing_success, jungle_where_predicates = parse_simple_logs(log_file_data)

Total log lines:  119488
Processed log_count:  10000
{'failure': 1,
 'failure_reason': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x1181ae790>, {}),
 'skipped': 6676,
 'skipped_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x1181ae820>, {}),
 'success': 3322,
 'success_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x10399e160>, {})}



Processed log_count:  20000
{'failure': 1,
 'failure_reason': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x1181ae790>, {}),
 'skipped': 13342,
 'skipped_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x1181ae820>, {}),
 'success': 6655,
 'success_log_type': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x10399e160>, {})}



Processed log_count:  30000
{'failure': 1,
 'failure_reason': defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x1181ae790>, {}),
 'skipped': 20008,
 'skipped_log_type': defaultdict(<function pars

Not supported query type: ALTER SYSTEM SET log_destination='stderr'
Not supported query type: ALTER SYSTEM SET logging_collector='off'
Not supported query type: ALTER SYSTEM SET log_statement='none'
Not supported query type: ALTER SYSTEM SET log_connections='off'
Not supported query type: ALTER SYSTEM SET log_disconnections='off'


In [3]:
jungle_where_predicates

defaultdict(<function column_usage.parse_simple_logs.<locals>.<lambda>()>,
            {(('jungle.int_field1', None), ('jungle.int_field1', None)): 10378,
             (('jungle.int_field4', None), ('jungle.int_field4', None)): 10396,
             (('jungle.int_field9', None), ('jungle.int_field9', None)): 10464,
             (('jungle.int_field4', None),
              ('jungle.int_field4', None),
              ('jungle.float_field9', None)): 274,
             (('jungle.int_field9', None),
              ('jungle.int_field9', None),
              ('jungle.float_field1', None)): 247,
             (('jungle.int_field9', None),
              ('jungle.int_field9', None),
              ('jungle.float_field9', None)): 263,
             (('jungle.int_field4', None),
              ('jungle.int_field4', None),
              ('jungle.float_field1', None)): 267,
             (('jungle.int_field9', None),
              ('jungle.int_field9', None),
              ('jungle.float_field2', None)): 237,


In [5]:
len(jungle_where_predicates)

34

In [13]:
pprint.pprint(jungle_where_predicates)

defaultdict(<function parse_simple_logs.<locals>.<lambda> at 0x1181ae8b0>,
            {(('jungle.int_field1', None), ('jungle.int_field1', None)): 10378,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.float_field0', None)): 243,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.float_field1', None)): 285,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.float_field2', None)): 246,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.float_field3', None)): 275,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.float_field4', None)): 293,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.float_field5', None)): 254,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.float_field6', None)): 257,
             (('jungle.int_field1', None), ('jungle.int_field1', None), ('jungle.f

In [14]:
pprint.pprint(generate_all_indexes(jungle_where_predicates)['jungle'])

{('float_field0', 'int_field1'),
 ('float_field0', 'int_field4'),
 ('float_field0', 'int_field9'),
 ('float_field1', 'int_field1'),
 ('float_field1', 'int_field4'),
 ('float_field1', 'int_field9'),
 ('float_field2', 'int_field1'),
 ('float_field2', 'int_field4'),
 ('float_field2', 'int_field9'),
 ('float_field3', 'int_field1'),
 ('float_field3', 'int_field4'),
 ('float_field3', 'int_field9'),
 ('float_field4', 'int_field1'),
 ('float_field4', 'int_field4'),
 ('float_field4', 'int_field9'),
 ('float_field5', 'int_field1'),
 ('float_field5', 'int_field4'),
 ('float_field5', 'int_field9'),
 ('float_field6', 'int_field1'),
 ('float_field6', 'int_field4'),
 ('float_field6', 'int_field9'),
 ('float_field7', 'int_field1'),
 ('float_field7', 'int_field4'),
 ('float_field7', 'int_field9'),
 ('float_field8', 'int_field1'),
 ('float_field8', 'int_field4'),
 ('float_field8', 'int_field9'),
 ('float_field9', 'int_field1'),
 ('float_field9', 'int_field4'),
 ('float_field9', 'int_field9'),
 ('int_fie

In [9]:
len(generate_all_indexes(jungle_where_predicates)['jungle'])

34

In [11]:
len(prune_indexes(generate_all_indexes(jungle_where_predicates))['jungle'])

34

In [ ]:
generate_create_index_commands(prune_indexes(generate_all_indexes(jungle_where_predicates)))

# TODOs:

- Some columns do not have table prefixed (rating vs review.rating)
- Fix column usage; is it even required?